# Tareas de SPSI
> Ana Buendía Ruiz-Azuaga
>
> Paula Villanueva Núñez

## Tarea 1
**Algoritmo extendedido de Euclides**

In [1]:
# not(m & 1) = ~m&1 -> Comprueba si el número es par. Devuelve true si es par

def bxeuc(m,n):
    '''
    Algoritmo extendido de euclides.
    Parametros: m,n dos enteros
    Devuelve: mcd y los coeficientes de Bezout
    '''
    shift = 0
    
    if m == 0:
        return n, 0, 1
    if n == 0:
        return m, 1, 0
    
    # Gestionamos numeros negativos
    signo_m = 1
    signo_n = 1
    
    if m < 0:
        m = -m
        signo_m = -1
    if n < 0:
        n = -n
        signo_n = -1
    
    
    while ~m&1 and ~n&1:
        m >>= 1
        n >>= 1
        
        shift += 1 
        
    # m0 y n0 son m y n quitando el factor comun 2, por ejemplo 4,2 pasa a 2,1
    m0 = m
    n0 = n
    
    sm = 1
    sn = 0
    
    tm = 0
    tn = 1
    
    # Se mantienen invariantes m = sm*m+sn*n, n=tm*m+tn*n
    
    
    while ~m&1: # Si m es par
        if not (~sm&1 and ~sn&1 ): # Si sm o sn alguno no es par
            # garantizamos que sm y sn sean pares
            sm = sm - n0
            sn = sn + m0
            
        # Quitamos el factor comun 2 de m, sm y sn, luego la identidad m = sm*m+sn*n se mantiene
        m >>= 1
        sm >>= 1
        sn >>= 1
    
    while n != 0:
        while ~n&1: # SI n par
            if not (~tm&1 and ~tn&1): # Si tm o tn alguno no es par
                # Aseguramos que tm y tn son pares
                tm = tm - n0
                tn = tn + m0
                
            # Eliminamos el factor comun 2 de n, tm y tn, luego la identidad n=tm*m+tn*n se mantiene
            n >>= 1
            tm >>= 1
            tn >>= 1
        
        # Si m>n intercambiamos las variables
        if m > n: 
            m, n, sm, tm, sn, tn = n, m, tm, sm, tn, sn
            
        
        # Vamos quitando tantas veces m a n
        n = n - m
        tm = tm - sm
        tn = tn - sn
        
    g = m << shift
    u = signo_m*sm
    v = signo_n*sn

    return g, u, v
    
    

In [14]:
m = 693
n = 609

g, u, v = bxeuc(m, n)

print("Máximo común divisor obtenido por el algoritmo: {}".format(g))
print("Resultado de la identidad usando los coeficientes {}".format(u*m+v*n))

print(g == u*m+v*n)

Máximo común divisor obtenido por el algoritmo: 21
Resultado de la identidad usando los coeficientes 21
True


## Tarea 2
**Máquina Enigma**

In [3]:
import sys

In [4]:
ANILLO_BASE = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [5]:
class Rotor:
    def __init__(self, notch, ringstellung, wiring):
        '''
        Constructor de la clase Rotor, recibe el notch, ringstelling y wiring de un rotor
        '''
        self.anillo = ANILLO_BASE
        self.notch = notch
        self.ringstellung = ringstellung
        self.wiring = wiring

        
    def aplicar_ringstellung(self, ringstellung):
        '''
        Metodo para aplicar el ringstellung al rotor
        '''
        indice_anillo = ANILLO_BASE.find(ringstellung)
        
        self.wiring = list(self.wiring)
        
        # Vamos reemplazando el wiring
        for i in range (0, len(self.wiring)):
            indice = ANILLO_BASE.find(self.wiring[i]) + indice_anillo
            self.wiring[i] = ANILLO_BASE[indice % 26]
            
        # Convertimos el wiring a cadena de nuevo   
        cadena = ""
        for c in self.wiring:
            cadena = cadena + c
        
        self.wiring = cadena
        # Colocamos el wiring para que empiece por la posicion adecuada
        self.wiring = self.wiring[-indice_anillo:]+self.wiring[:-indice_anillo]
        
        
        self.ringstellung = ringstellung
        
    
    def aplicar_grundstellung(self, grundstellung):
        '''
        Metodo para aplicar grundstellung al rotor
        '''
        
        # Estar en la ventanita es estar al principio de la cadena
        indice = self.anillo.find(grundstellung)
        
        self.anillo = self.anillo[indice:] + self.anillo[:indice]
        self.wiring = self.wiring[indice:] + self.wiring[:indice]
        

    def cifrar_caracter(self, car):
        '''
        Metodo para cifrar un caracter con un rotor
        Devuelve el caracter cifrado
        '''
        # Guardo posicion de la letra en el alfabeto
        indice = ANILLO_BASE.find(car)
        
        # Accedemos a wiring de la letra
        letra = self.wiring[indice]
        
        # Convertimos la letra segun corresponde en wiring
        ind = self.anillo.find(letra)
        
        return ANILLO_BASE[ind]
    
    def inv_cifrar_caracter(self, car):
        '''
        Metodo para cifrar un caracter en sentido inverso usando un rotor
        Devuelve el caracter cifrado
        '''
        # Guardo posicion de la letra en el alfabeto
        indice = ANILLO_BASE.find(car)
        
        # Accedemos a wiring de la letra
        letra = self.anillo[indice]
        
        # Convertimos la letra segun corresponde en wiring
        ind = self.wiring.find(letra)
        
        return ANILLO_BASE[ind]
    
    def rotar(self):
        '''
        Metodo para rotar un rotor una posicion
        '''
        # Avanzo anillo y wiring una posicion
        self.anillo = self.anillo[1:] + self.anillo[:1]
        self.wiring = self.wiring[1:] + self.wiring[:1]

    
    def comprobar_vuelta_completa(self):
        '''
        Metodo que comprueba si el rotor ha dado una vuelta completa.
        Devuelve true si la ha dado, false si no
        '''
        # Dar vuelta completa es que el caracter de la ventana esté en el notch
        return self.anillo[0] in self.notch


In [6]:
ROTOR_I = {"notch": "Q", "ringstellung": "A", "wiring": "EKMFLGDQVZNTOWYHXUSPAIBRCJ"}
    
ROTOR_II = {"notch": "E", "ringstellung": "A", "wiring": "AJDKSIRUXBLHWTMCQGZNPYFVOE"}
    
ROTOR_III = {"notch": "V", "ringstellung": "A", "wiring": "BDFHJLCPRTXVZNYEIWGAKMUSQO"}
    
ROTOR_IV = {"notch": "J", "ringstellung": "A", "wiring": "ESOVPZJAYQUIRHXLNFTGKDCMWB"}
    
ROTOR_V = {"notch": "Z", "ringstellung": "A", "wiring": "VZBRGITYUPSDNHLXAWMJQOFECK"}
    
ROTOR_VI = {"notch": "ZM", "ringstellung": "A", "wiring": "JPGVOUMFYQBENHZRDKASXLICTW"}
    
ROTOR_VII = {"notch": "ZM", "ringstellung": "A", "wiring": "NZJHGRCXMYSWBOUFAIVLPEKQDT"}
    
ROTOR_VIII = {"notch": "ZM", "ringstellung": "A", "wiring": "FKQHTLXOCBJSPDZRAMEWNIUYGV"}
    
ROTOR_BETA = {"notch": "", "ringstellung": "A", "wiring": "LEYJVCNIXWPBQMDRTAKZGFUHOS"}
    
ROTOR_GAMMA = {"notch": "", "ringstellung": "A", "wiring": "FSOKANUERHMBTIYCWLQPZXVGJD"}

In [7]:
class Reflector:
    
    def __init__(self, config_reflector):
        '''
        Constructor de la clase Reflector, recibe el wiring del reflector
        '''
        self.reflector = list(config_reflector)
        
    def reflejar(self, car):
        '''
        Metodo para reflejar (cifrar) un caracter usando el reflector
        '''
        indice = ANILLO_BASE.find(car)
        
        # Accedemos a wiring de la letra
        letra = self.reflector[indice % 26]
        
        # Convertimos la letra segun corresponde en wiring
        ind = ANILLO_BASE.find(letra)
        return ANILLO_BASE[ind % 26]

In [8]:
REFLECTOR_B = {"reflector": "YRUHQSLDPXNGOKMIEBFZCWVJAT"}
    
REFLECTOR_C = {"reflector": "FVPJIAOYEDRZXWGCTKUQSBNMHL"}
    
REFLECTOR_B_THIN = {"reflector": "ENKQAUYWJICOPBLMDXZVFTHRGS"}
    
REFLECTOR_C_THIN = {"reflector": "RDOBJNTKVEHMLFCWZAXGYIPSUQ"}

In [9]:
class Plugboard:
    def __init__(self, sustituciones):
        '''
        Constructor de PLugboard, recibe una lista de las sustituciones en el teclado
        '''
        parejas = sustituciones.split(" ")
        
        self.mapa = {}
        
        # Asignamos cada letra consigo misma
        for letra in ANILLO_BASE:
            self.mapa[letra] = letra
        # Intercambiamos las letras indicadas
        for elem in parejas:
            self.mapa[elem[0]] = elem[1]
            self.mapa[elem[1]] = elem[0]
    
    def sustituir_letra(self, car):
        '''
        Metodo para sustituir una letra por su letra correspondiente
        '''
        return self.mapa[car]

In [10]:
class MaquinaEnigmaM4:
    '''Clase para modelar máquina enigma  Wehrmacht con posibilidad de simular una Kriegsmarine'''
    
    def __init__(self, reflector, rotores, ringstellung, grundstellung, clavijas):
        '''Constructor de la clase con el modelo de la maquina, configuracion inicial de los rotores y reflector'''
        
        if len(rotores) != 4 or len(rotores) != len(ringstellung) or len(rotores) != len(grundstellung):
                sys.exit("Error, la máquina Kriegsmarine M4 no puede tener un número de rotores distinto de 4 o las especiifcaciones de ringstellung y grundstellung no se corresponden con los rotores ")
        
        # Invierto vectores para adaptarlo a la implementacion
        rotores = rotores[::-1]
        ringstellung = ringstellung[::-1]
        grundstellung = grundstellung[::-1]
        
        
        # Añado rotores a maquina enigma
        self.rotores = []
        for rotor in rotores:
            self.rotores.append(Rotor(rotor['notch'], rotor['ringstellung'], rotor['wiring']))
        
        # Añado ringstellung y lo aplico a cada rotor
        self.ringstellung = ringstellung
        
        for i in range(0, len(ringstellung)):
            self.rotores[i].aplicar_ringstellung(self.ringstellung[i])
        
        # Añado grunstellung y se aplica a cada rotor
        self.grundstellung = grundstellung
        
        for i in range(0, len(grundstellung)):
            self.rotores[i].aplicar_grundstellung(self.grundstellung[i])
        
        # Añado clavijas
        self.clavijas = clavijas
        
        self.plugboard = Plugboard(self.clavijas)
        
        # Añado reflector
        self.reflector = Reflector(reflector['reflector'])
    
    
    def cifrar_caracter(self, car):
        '''
        Metodo para cifrar un carater usando la maquina enigma
        '''
        # Aplico plugboard
        car = self.plugboard.sustituir_letra(car)
        
        # Comprobar si los rotores necesitan girar. El ultimo rotor no gira
        girar = [True, False, False, False]
        
        for i in range(0, len(girar)-1):
            if self.rotores[i].comprobar_vuelta_completa():
                girar[i+1] = True
                
        for i in range(0, len(girar)-1):
            if girar[i]:
                self.rotores[i].rotar()
    
        # Rotación doble del segundo rotor
        if girar[2]:
            self.rotores[1].rotar()
        
        # Cifro caracter en cada rotor
        for rotor in self.rotores:
            car = rotor.cifrar_caracter(car)
            
        # Reflejo caracter
        car = self.reflector.reflejar(car)
        
        # Cifro caracter inversamente en cada rotor
        for rotor in reversed(self.rotores):
            car = rotor.inv_cifrar_caracter(car)
        
        # Aplico plugboard
        car = self.plugboard.sustituir_letra(car)
        
        return car
    
    def cifrar(self, texto):
        '''
        Metodo para cifrar un texto
        '''
        texto_cifrado = ""
        for car in texto:
            if (car != " "):
                texto_cifrado = texto_cifrado + self.cifrar_caracter(car)
            
        return texto_cifrado
    
    
        

## Tarea 3

**Descifrando el texto con la máquina enigma M4**

In [11]:
maquina = MaquinaEnigmaM4(REFLECTOR_C_THIN, [ROTOR_BETA, ROTOR_V, ROTOR_VI, ROTOR_VIII], "EPEL", "NAEM", "AE BF CM DQ HU JN LX PR SZ VW")

nuevo_grundstellung = "QEOB"
grundstellung = maquina.cifrar(nuevo_grundstellung)

# Configuramos la nueva Enigma con el grundstellung que hemos obtenido
enigma = MaquinaEnigmaM4(REFLECTOR_C_THIN, [ROTOR_BETA, ROTOR_V, ROTOR_VI, ROTOR_VIII], "EPEL", grundstellung, "AE BF CM DQ HU JN LX PR SZ VW")

mensaje = "DUHF TETO LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW DUHF TETO"

# Quitar 8 ultimos caracteres y 8 primeros, con espacios   
mensaje = mensaje[10:len(mensaje)-10]

mensaje_cifrado = enigma.cifrar(mensaje)


print(f"\nMensaje original:\n {mensaje}")
print(f"\nMensaje cifrado:\n {mensaje_cifrado}")


Mensaje original:
 LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW

Mensaje cifrado:
 KRKRALLEXXFOLGENDESISTSOFORTBEKANNTZUGEBENXXICHHABEFOLGELNBEBEFEHLERHALTENXXJANSTERLEDESBISHERIGXNREICHSMARSCHALLSJGOERINGJSETZTDERFUEHRERSIEYHVRRGRZSSADMIRALYALSSEINENNACHFOLGEREINXSCHRIFTLSCHEVOLLMACHTUNTERWEGSXABSOFORTSOLLENSIESAEMTLICHEMASSNAHMENVERFUEGENYDIESICHAUSDERGEGENWAERTIGENLAGEERGEBENXGEZXREICHSLEITEIKKTULPEKKJBORMANNJXXOBXDXMMMDURNHFKSTXKOMXADMXUUUBOOIEXKP


### Cómo es compatible con la Máquina Enigma M3

La M4 configurada con el reflector B_Thin y rueda griega Beta en posición A y Ringstellung A, simula la M3 con reflector B. Por otra parte si es configurada con el reflector C_Thin y rueda griega Gamma en posición A y Ringstellung A, simula la M3 con reflector C.

## Tarea 4

**Protocolo de uso de la enigma que haga que el receptor sepa a quién de su unidad debe dirigir el mensaje recibido, que no sea capaz de leer su contenido y que sin embargo el oficial receptor sí pueda hacerlo por sus propios medios**